**disallowedDeprecatedContent.ipynb**

Deprecated concept with disallowed annotations or axioms.

**Documentation:** https://github.com/edamontology/edamverify/blob/master/docs/disallowedDeprecatedContent.md

**NB.1: - Running the notebook**
The directory containing the ``EDAM_dev.owl`` file must be defined by ``EDAM_PATH`` environment variable.

The script requires the test to be run from a subdirectory of ``EDAM_PATH`` (hence ``'../EDAM_dev.owl'`` below)

In [17]:
import os
from rdflib import ConjunctiveGraph, Namespace
import json

# Constants for script error reporting as per https://github.com/edamontology/edamverify.
NOERR = "NOERR"
INFO  = "INFO"
WARN  = "WARN"
ERROR = "ERROR"

#Load EDAM_dev.owl from GitHub into an RDF graph.
print("Loading graph ...", end="")
g = ConjunctiveGraph()
g.load(os.environ.get('EDAM_PATH', '../EDAM_dev.owl'), format='xml')
# g.load('https://raw.githubusercontent.com/edamontology/edamontology/master/EDAM_dev.owl', format='xml')
# g.load('EDAM_dev.owl', format='xml')
g.bind('edam', Namespace('http://edamontology.org#'))
print("done!")



Loading graph ...done!


Define SPARQL query to retrieve ID, term, and all properties that are *not* allowed on deprecated concepts. Run the query.

**NB.1:** BASE is used to define the define the default namespace (for various elements below).

**NB.2:** A concept is considered deprecated if it has a property ``<oboInOwl:inSubset>`` whose value contains the string ``obsolete``, e.g. ``<oboInOwl:inSubset rdf:resource="http://purl.obolibrary.org/obo/edam#obsolete"/>`` or
``<oboInOwl:inSubset>obsolete />``

In [18]:
# Compile SPARQL query
query_term = """
BASE <http://edamontology.org/>
SELECT ?id ?term ?subsetpath ?documentation ?exact_syn ?narrow_syn ?broad_syn ?notRecommendedForAnnotation ?file_extension ?media_type ?example ?information_standard ?ontology_used ?organisation ?regex WHERE
{
?id rdfs:label ?term .
?id oboInOwl:inSubset ?subsetpath .
OPTIONAL {?id :documentation ?documentation .}
OPTIONAL {?id oboInOwl:hasExactSynonym ?exact_syn .}
OPTIONAL {?id oboInOwl:hasNarrowSynonym ?narrow_syn .}
OPTIONAL {?id oboInOwl:hasBroadSynonym ?broad_syn .}
OPTIONAL {?id rdfs:comment ?comment .}
OPTIONAL {?id :notRecommendedForAnnotation ?notRecommendedForAnnotation .}
OPTIONAL {?id :file_extension ?file_extension .}
OPTIONAL {?id :media_type ?media_type .}
OPTIONAL {?id :example ?example .}
OPTIONAL {?id :information_standard ?information_standard .}
OPTIONAL {?id :ontology_used ?ontology_used .}
OPTIONAL {?id :organisation ?organisation .}
OPTIONAL {?id :regex ?regex .}
}
"""

# Run SPARQL query and collate results
errfound = False
report = list()
results = g.query(query_term)

Analyse results of query.

In [19]:
# Error message building function
def report_append(msg):
    """
    Writes a report of an error and sets the error level appropriately.
    err: the error level to set
    msg: the message to write
    """
    global errfound
    global report

    report.append("Invalid property " + "(" + msg + ") " + "on ::: " + id +  " (" + term + ")")
    errfound = True

for r in results :
#    print(str(r['id']), str(r['term']), str(r['ext']), str(r['exact_syn']))
    id   = str(r['id'])
    term = str(r['term'])
    subsetpath  = str(r['subsetpath'])

    documentation = str(r['documentation'])
    exact_syn = str(r['exact_syn'])
    narrow_syn = str(r['narrow_syn'])
    broad_syn = str(r['broad_syn'])
    notRecommendedForAnnotation = str(r['notRecommendedForAnnotation'])
    file_extension = str(r['file_extension'])
    media_type = str(r['media_type'])
    example = str(r['example'])
    information_standard = str(r['information_standard'])
    ontology_used = str(r['ontology_used'])
    organisation = str(r['organisation'])
    regex = str(r['regex'])

    # Check for obsolete concepts
    if "obsolete" in subsetpath:
        if documentation != "None":
            report_append("documentation")
        if exact_syn  != "None":
            report_append("exact_syn")
        if narrow_syn  != "None":
            report_append("narrow_syn")
        if broad_syn  != "None":
            report_append("broad_syn")
        if notRecommendedForAnnotation != "None":
            report_append("notRecommendedForAnnotation")
        if file_extension  != "None":
            report_append("file_extension")
        if media_type  != "None":
            report_append("media_type")
        if example  != "None":
            report_append("example")
        if information_standard != "None":
            report_append("information_standard")
        if ontology_used  != "None":
            report_append("ontology_used")
        if organisation  != "None":
            report_append("organisation")
        if regex != "None":
            report_append("regex")

Write report and return approriate value.

In [20]:
report_obj = {}
report_obj['test_name'] = 'disallowedDeprecatedContent'
report_obj['comment'] = 'Invalid property found for one or more obsolete concepts.'

if errfound:
    report_obj['status'] = WARN
    report_obj['reason'] = report
else:
    report_obj['status'] = NOERR

report_json = json.dumps(report_obj, indent=4)
print(report_json)

{
    "comment": "Invalid property found for one or more obsolete concepts.",
    "status": "NOERR",
    "test_name": "disallowedDeprecatedContent"
}
